In [1]:
import pandas as pd
import numpy as np
#import seaborn as sns
#import pickle
#import os
#import matplotlib.pyplot as plt
#import math as mp
#from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
from sklearn.metrics import make_scorer

def find_threshold_for_efficiency(a, e, w):
    if e < 0 or e > 1:
        raise ValueError("Efficiency e must be in [0, 1]")
    # Decreasing order
    idx = np.argsort(a)[::-1]
    a_sort = a[idx]
    if w is None:
        w = np.ones(a.shape)
    w_sort = w[idx]
    ecdf = np.cumsum(w_sort)
    if (ecdf[-1]) <= 0:
        raise ValueError("Total weight is < 0")

    target_weight_above_threshold = e * ecdf[-1]
    enough_passing = ecdf >= target_weight_above_threshold
    first_suitable = np.argmax(enough_passing)
    last_unsuitable_inv = np.argmin(enough_passing[::-1])
    if last_unsuitable_inv == 0:
        raise ValueError("Bug in code")
    last_unsuitable_plus = len(a) - last_unsuitable_inv
    return 0.5*(a_sort[first_suitable] + a_sort[last_unsuitable_plus])


def get_rejection_at_efficiency_raw(
        labels, predictions, weights, quantile):
    signal_mask = (labels >= 1)
    background_mask = ~signal_mask
    if weights is None:
        signal_weights = None
    else:
        signal_weights = weights[signal_mask]
    threshold = find_threshold_for_efficiency(predictions[signal_mask], 
                                              quantile, signal_weights)
    rejected_indices = (predictions[background_mask] < threshold)
    if weights is not None:
        rejected_background = weights[background_mask][rejected_indices].sum()
        weights_sum = np.sum(weights[background_mask])
    else:
        rejected_background = rejected_indices.sum()
        weights_sum = np.sum(background_mask)
    return rejected_background, weights_sum         


def get_rejection_at_efficiency(labels, predictions, threshold, sample_weight=None):
    rejected_background, weights_sum = get_rejection_at_efficiency_raw(
        labels, predictions, sample_weight, threshold)
    return rejected_background / weights_sum


def rejection90(labels, predictions, sample_weight=None):
    return get_rejection_at_efficiency(labels, predictions, 0.9, sample_weight=sample_weight)


rejection90_sklearn = make_scorer(
get_rejection_at_efficiency, needs_threshold=True, threshold=0.9)

In [2]:
%%time
df = pd.read_csv('train_part_1_v2.csv')
#df = df[:int(len(df)/2)]
df = df.replace(-9999, pd.np.nan)
df = df.dropna(how = 'any')
#df = pd.read_csv('qwerty.csv')

CPU times: user 1min 1s, sys: 7.19 s, total: 1min 8s
Wall time: 1min 16s


In [4]:
df.columns[df.nunique() == 1]

MemoryError: 

In [ ]:
(np.std(df['MatchedHit_Z[0]']),
np.mean(df['MatchedHit_Z[0]']), np.mean(df['MatchedHit_Z[3]']))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(df['MatchedHit_Z[0]'], bins=100)

In [ ]:
# проверка прямолинености полета
delta = df['Lextra_X[3]'] - df['Lextra_X[0]']
dif = delta/3 - df['Lextra_X[1]']
del delta 
del dif

In [ ]:
# проверка прямолинейности для реальных
delta_x = df['MatchedHit_X[1]'] - df['MatchedHit_X[0]']
delta_y = df['MatchedHit_Y[1]'] - df['MatchedHit_Y[0]']

radius = (delta_x**2 + delta_y**2)**0.5
#dif_r = delta_r - df['MatchedHit_X[1]']
#dif_r.mean()

In [ ]:
delta1_x = df['MatchedHit_X[1]'] - df['MatchedHit_X[0]']
delta1_y = df['MatchedHit_Y[1]'] - df['MatchedHit_Y[0]']

radius1 = (delta1_x**2 + delta1_y**2)**0.5

In [ ]:
del delta1_y

In [ ]:
delta2_x = df['MatchedHit_X[2]'] - df['MatchedHit_X[1]']
delta2_y = df['MatchedHit_Y[2]'] - df['MatchedHit_Y[1]']

radius2 = (delta2_x**2 + delta2_y**2)**0.5

In [ ]:
#dt = kf['MatchedHit_T[3]'] - kf['MatchedHit_T[0]']
#print(len(df[(radius1 < radius1.std())]))
#len(df[(radius1 > (radius1.mean() - radius1.std()) ) & (radius1 < (radius1.mean() + radius1.std()) ) ])
#kf[kf.label == 1]['MatchedHit_T[1]'].std()
kf[kf.label == 1].PT.std()

In [4]:
#kf = df[(radius1 < radius1.std())]
#kf = df[(radius1 > (radius1.mean() - radius1.std()) ) & (radius1 < (radius1.mean() + radius1.std()) ) ]
#kf = df[df['NShared'] != 0]
kf = df[df['MatchedHit_T[0]'] < df['MatchedHit_T[0]'].std()]
#kf = kf[kf['MatchedHit_T[1]'] < kf['MatchedHit_T[1]'].std()]
#kf = kf[kf['MatchedHit_T[2]'] < kf['MatchedHit_T[2]'].std()]


#kf = kf[kf['NShared'] != 0]

ratio = float(np.sum(kf.label == 1)) / np.sum(kf.label == 0)
#sample_size = sum(kf.label == 0)  # Equivalent to len(data[data.Healthy == 0])
#random_indices = np.random.choice(healthy_indices, sample_size, replace=False)

#kf = kf.loc[random_indices]

#& (df['MatchedHit_TYPE[0]'] == 2)

In [ ]:
len(kf)

In [ ]:
#back_index = kf[kf.label == 0]
#signal_index = kf[kf.label == 1][:len(back_index)]
healthy_indices = kf[kf.label == 1].index
back = kf[kf.label == 0].index

#healthy_indices = data[data.Healthy == 1].index
random_signal = np.random.choice(healthy_indices, len(back), replace=False)
signal_sample = kf.loc[random_signal]


#healthy_indices = data[data.Healthy == 1].index
random_back = np.random.choice(back, len(back), replace=False)
back_sample = kf.loc[random_back]



healthy = pd.concat([back_sample, signal_sample])
healty.to()

In [ ]:
len(kf[kf['MatchedHit_TYPE[3]'] == 2])


In [ ]:
df[(radius2 < radius2.std()) & (df['MatchedHit_TYPE[3]'] == 2)].label.value_counts()

In [5]:
y = kf.label
X  = kf.drop(['id','sWeight','particle_type','label','kinWeight','weight','FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S'], axis = 1)

In [ ]:
del kf

In [6]:
import xgboost as xgb

In [7]:
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

In [ ]:
ratio

In [8]:
params = {
    'objective':'binary:logistic',
    'n_jobs':-1,
    'max_depth': 3,
    'nthread': 4,
    'silent': 1,
    'min_child_weight': 6,
    'ratio': 20
}
num_rounds = 20

In [9]:
#weights = np.zeros(len(y))
#weights[y == 0] = 20
#weights[y == 1] = 1

dtrain = xgb.DMatrix(X, label=y) # weights added
#dtest = xgb.DMatrix(X_test)

In [10]:
%%time
xgb_model = xgb.train(params, dtrain, num_rounds)


#y_test_preds = (xgb_model.predict(X_test) > 0.5).astype('int')

CPU times: user 16.7 s, sys: 97.5 ms, total: 16.8 s
Wall time: 4.48 s


In [ ]:
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

In [ ]:
rejection90(y_test, xgb_model.predict(xgb.DMatrix(X_test)))

In [ ]:
#f = df.copy()
y = df.label
X  = df.drop(['id','sWeight','particle_type','label','kinWeight','weight','FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S'], axis = 1)

import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]

In [11]:
del df

In [ ]:
X = X[X['MatchedHit_T_0_'] < X['MatchedHit_T_0_'].std()]

In [ ]:
y = y[X.index]

In [12]:
ll = pd.read_csv('test_public_v2.csv')
test_y = ll.id


In [13]:
test_X = ll.drop(['id','FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S'], axis = 1)

In [15]:
regex = re.compile(r"\[|\]|<", re.IGNORECASE)

test_X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]
pred = xgb_model.predict(xgb.DMatrix(test_X))

In [16]:
#predicted_df = pd.DataFrame([test_y,pred], index = np.arange(1, pred.shape[0] + 1))
predicted_df = pd.DataFrame(data={"prediction": pred, "id": test_y}) 
predicted_df.to_csv("sample_submission.csv", index_label='id')
predicted_df.info()
#, "id": test_y
#predicted_df.to_csv('ans.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 726095 entries, 0 to 726094
Data columns (total 2 columns):
prediction    726095 non-null float32
id            726095 non-null int64
dtypes: float32(1), int64(1)
memory usage: 8.3 MB


In [ ]:
predicted_df.head()

In [ ]:
rejection90(y_test, xgb_model.predict(dtest))

In [ ]:
rejection90(y, xgb_model.predict(xgb.DMatrix(X)))

In [ ]:
#xgb.plot_tree
#xgb.plot_importance(xgb_model);

In [ ]:
from hyperopt import hp
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

In [ ]:
weights = np.zeros(len(y_train))
weights[y_train == 0] = 15
weights[y_train == 1] = 1

dtrain = xgb.DMatrix(X_train, label=y_train, weight=weights)
dvalid = xgb.DMatrix(X_test, label=y_test)

In [ ]:
def score(params):
    print("Training with params:")
    print(params)
    params['max_depth'] = int(params['max_depth'])
    #dtrain = xgb.DMatrix(X_train, label=y_train, weight=weights)
    #dvalid = xgb.DMatrix(X_test, label=y_test)
    model = xgb.train(params, dtrain, params['num_round'])
    predictions = model.predict(dvalid) #.reshape((X_test.shape[0], 7))
    score = rejection90(y_test, predictions)
    print("\tScore {0}\n\n".format(score))
    return {'loss': score, 'status': STATUS_OK}

In [ ]:
def optimize(trials):
    space = {
             'num_round': 100,
             'learning_rate': hp.quniform('eta', 0.005, 0.05, 0.005),
             'max_depth': 3,
             'min_child_weight': 6,
             'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
             'gamma': hp.quniform('gamma', 0.5, 1, 0.01),
             'colsample_bytree': hp.quniform('colsample_bytree', 0.4, 1, 0.05),
             'eval_metric': 'error',
             'objective': 'binary:logistic',
             'nthread' : 4,
             'silent' : 0,
             'n_jobs' : -1
             }
    
    best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=10)
    return best


In [ ]:
%%time
trials = Trials()
best_params = optimize(trials)
best_params

In [ ]:
best_params
0.551223186742234
0.552442786426573

In [ ]:
#best_params['max_depth'] = 3
#best_params['eval_metric'] = 'merror'
#best_params['objective'] = 'binary:logistic'
#best_params['nthread'] = 4
#best_params['silent'] = 1
#best_params['n_jobs'] =  -1
#best_params['colsample_bytree'] = 0.9
#best_params['eta'] = 0.02
#best_params['gamma'] = 0.58
#best_params['min_child_weight'] = 6
#best_params['subsample'] = 0.6000000000000001

best_params = {
    'num_round': 100,
    'objective':'binary:logistic',
    'n_jobs':-1,
    'max_depth': 3,
    'eval_metric': 'error',
    'nthread': 4,
    'silent': 1,
    'colsample_bytree': 0.9,
    'eta': 0.02,
    'gamma': 0.58,
    'min_child_weight': 6,
    'subsample': 0.6000000000000001
}



In [ ]:
#dtrain = xgb.DMatrix(X_train, y_train)

In [ ]:
weights = np.zeros(len(y_train))
weights[y_train == 0] = 20 #11
weights[y_train == 1] = 1

dtrain = xgb.DMatrix(X_train, label=y_train, weight=weights)
dtest = xgb.DMatrix(X_test, label=y_test)

In [ ]:
num_rounds = 20
#bestXgb = xgb.train(best_params, dtrain, num_rounds, feval = rejection90, maximize = True)
bestXgb = xgb.XGBClassifier()
bestXgb.fit(X_train, y_train)

#xgboost_predict_proba = bestXgb.predict(dtest)
#xgboost_prediction = np.argmax(xgboost_predict_proba,axis=1)

In [ ]:
#dtest = xgb.DMatrix(y_test)
#rejection90(y_test, xgb_model.predict(dtest))
rejection90(y_test, bestXgb.predict(dtest))

In [ ]:
bestXgb.predict(dvalid).shape

In [ ]:
model = xgb.XGBClassifier(nthread = 4, n_jobs = -1)
model.fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
rejection90(y_test, model.predict(X_test))

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model = ExtraTreesClassifier()

model.fit(X, y)

print(model.feature_importances_)

In [ ]:
pion = df[df.particle_type == 0]
muon = df[df.particle_type == 1]
proton = df[df.particle_type == 2]

In [ ]:
muon[['MatchedHit_DT[3]','MatchedHit_DT[2]','MatchedHit_DT[1]','MatchedHit_DT[0]','MatchedHit_T[3]', 'MatchedHit_T[2]', 'MatchedHit_T[1]', 'MatchedHit_T[0]']].corr()

In [ ]:
speed_pion_2 = pion['MatchedHit_DT[2]'] - pion['MatchedHit_DT[1]']

In [ ]:
len(speed_pion_2)

In [ ]:
speed_pion_2[speed_pion_2 == 0].count()

In [ ]:
muon['MatchedHit_DT[3]'].mean()

In [ ]:
proton['MatchedHit_DT[3]'].mean()

In [ ]:
len(muon[pion['MatchedHit_X[3]'] == -9999])

In [ ]:
#df1 = df[:int(len(df)/4)]

In [ ]:
#df2 = df1.copy()
#df2 = df2.replace(-9999, pd.np.nan)
#df2 = df2.dropna(how = 'any')
#df1[df1 == -0]

In [ ]:
#df2.shape

In [ ]:
#df3 = df2[:int(len(df2)/4)]
#df3.to_csv(path_or_buf = '/home/rebinew/python/yandex/qwerty.csv', index = False)

In [ ]:
dd = pd.read_csv('/home/rebinew/python/yandex/qwerty.csv')

In [ ]:
dd.info()

In [ ]:
pion = dd[dd.particle_type == 0]
muon = dd[dd.particle_type == 1]
proton = dd[dd.particle_type == 2]

In [ ]:
muon = muon.iloc[:len(proton)]

In [ ]:
pion = pion.iloc[:len(proton)]

In [ ]:
min(proton.kinWeight)

In [ ]:
max(muon.kinWeight)

In [ ]:
#test = dd[['particle_type', 'kinWeight', 'P']]
test = pd.concat([muon, pion, proton])

In [ ]:
#alpha_X01 = []
#mas = []
#for i in range(len(test)):
#    alpha_X01 = mp.fabs(test['MatchedHit_X[1]'] - test['MatchedHit_X[0]'])**2
#    mas.append(alpha_X01)

In [ ]:
#dd = dd.assign(ALPHA_X01 = mas)

In [ ]:
new_test = test.copy()

In [ ]:
dif = test['MatchedHit_X[1]'] - test['MatchedHit_X[0]']
#dist2 = dif**2 + 1500**2
#dist = dist2**0.5
#speed = dist / (test['MatchedHit_T[1]'])
new_test = new_test.assign(dif1 = dif)

In [ ]:
#для первой
dist = max(new_test.dif1) - min(new_test.dif1)
first_segment = new_test[(new_test.dif1**2)**0.5 <= dist/4]
second_segment = new_test[((new_test.dif1**2)**0.5 > dist/4) & ((new_test.dif1**2)**0.5 <= dist/2)]
third_segment = new_test[((new_test.dif1**2)**0.5 > dist/2) & ((new_test.dif1**2)**0.5 <= 3*dist/4)]
fourth_segment = new_test[(new_test.dif1**2)**0.5 > 3*dist/4]

In [ ]:
# dist = 130
dist

In [ ]:
print(len(first_segment))
print(len(second_segment))
print(len(third_segment))
print(len(fourth_segment))

In [ ]:
len(new_test[((new_test.dif1**2)**0.5 > 0.5825*dist) & ((new_test.dif1**2)**0.5 <= 1.1951*dist)])

In [ ]:
#dist = dist/4
first_segment = new_test[(new_test.dif1**2)**0.5 <= dist/4]
second_segment = new_test[((new_test.dif1**2)**0.5 > dist/4) & ((new_test.dif1**2)**0.5 <= 0.5825*dist)]
third_segment = new_test[((new_test.dif1**2)**0.5 > 0.5825*dist) & ((new_test.dif1**2)**0.5 <= 1.1951*dist)]
fourth_segment = new_test[(new_test.dif1**2)**0.5 > 1.1951*dist]

In [ ]:
sns.distplot(first_segment.particle_type, kde=False, rug=True, label = 'first')
sns.distplot(second_segment.particle_type, kde=False, rug=True, label = 'second')
sns.distplot(third_segment.particle_type, kde=False, rug=True, label = 'third')
sns.distplot(fourth_segment.particle_type, kde=False, rug=True, label = 'fourth')
plt.legend();

In [ ]:
c = first_segment.corr().abs()

s = c.unstack()
so = s.sort_values(kind="quicksort")

so['label']

In [ ]:
new_test = new_test.assign(first_segment = first_segment.dif1)
new_test = new_test.assign(second_segment = second_segment.dif1)
new_test = new_test.assign(third_segment = third_segment.dif1)
new_test = new_test.assign(fourth_segment = fourth_segment.dif1)

In [ ]:
sns.distplot(new_test[(new_test.particle_type == 0) & (new_test.dif1 > 0)]['MatchedHit_X[0]'], kde=False, rug=True)
sns.distplot(new_test[(new_test.particle_type == 1) & (new_test.dif1 > 0)]['MatchedHit_X[0]'], kde=False, rug=True)
sns.distplot(new_test[(new_test.particle_type == 2) & (new_test.dif1 > 0)]['MatchedHit_X[0]'], kde=False, rug=True);

In [ ]:
new_test[(new_test.particle_type == 0) & (new_test.dif1 > 0)].count()

In [ ]:
new_test[(new_test.particle_type == 2) & (new_test.dif1 == 0)].count()

In [ ]:
new_test[(new_test.particle_type == 1) & (new_test.dif1 < 0)].count()

In [ ]:
new_test[(new_test.particle_type == 1) & (new_test.dif1 > 0)].count()

In [ ]:
new_test[(new_test.particle_type == 2) & (new_test.dif1 < 0)].count()

In [ ]:
new_test[(new_test.particle_type == 2) & (new_test.dif1 > 0)].count()

In [ ]:
new_test[new_test.particle_type == 2].dif1.mean()

In [ ]:
y0 = []
y1 = []
for i in range(len(test)):
    y0.append(0)
    y1.append(1)

plt.plot(test['MatchedHit_X[0]'], y0, color = 'r')
plt.plot(test['MatchedHit_X[1]'], y1, color = 'g');

In [ ]:
a = new_test[new_test.particle_type == 1].speed1.mean()

In [ ]:
b = new_test[new_test.particle_type == 1].speed2.mean()

In [ ]:
c = new_test[new_test.particle_type == 1].speed3.mean()

In [ ]:
a = new_test[new_test.particle_type == 0].speed1.mean()
b = new_test[new_test.particle_type == 0].speed2.mean()
c = new_test[new_test.particle_type == 0].speed3.mean()

a1 = new_test[new_test.particle_type == 1].speed1.mean()
b1 = new_test[new_test.particle_type == 1].speed2.mean()
c1 = new_test[new_test.particle_type == 1].speed3.mean()

a2 = new_test[new_test.particle_type == 2].speed1.mean()
b2 = new_test[new_test.particle_type == 2].speed2.mean()
c2 = new_test[new_test.particle_type == 2].speed3.mean()

plt.plot([0,1,2],[a,b,c])
plt.plot([0,1,2],[a1,b1,c1])
plt.plot([0,1,2],[a2,b2,c2])

In [ ]:
#test.info()

In [ ]:
from sklearn.model_selection import train_test_split
y = test.label
test.drop(['FOI_hits_X','FOI_hits_Y','FOI_hits_Z','FOI_hits_DX','FOI_hits_DY','FOI_hits_DZ','FOI_hits_T','FOI_hits_DT','FOI_hits_S'], axis = 1, inplace = True)
#withDif = new_test[['particle_type','first_segment','second_segment','third_segment','fourth_segment']]
#withDif = withDif.dropna(how = 'any', axis = 1)
X_train, X_test, y_train, y_test = train_test_split(test, y, test_size=0.3, random_state=42)

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=0, n_jobs = -1, criterion = 'entropy', min_samples_split = 2, min_samples_leaf = 4, max_features = 8)
clf.fit(X_train, y_train)
print(clf.score(X_train, y_train))

In [ ]:
rejection90(y_train, clf.predict(X_train))

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
#clf = DecisionTreeClassifier(random_state=0)
#clf.fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100, max_depth=5,random_state=0)
rfc.fit(X_train, y_train)

In [ ]:
rfc.score(X_test, y_test)

In [ ]:
dd_0 = dd[dd.particle_type == 0]
dd_1 = dd[dd.particle_type == 1]
dd_2 = dd[dd.particle_type == 2] 

In [ ]:
dd.head()

In [ ]:
%%time
pion = df[df.particle_type == 0]
plt.plot(pion['Lextra_X[3]'], pion['Lextra_Y[3]'], 'o')

In [ ]:
%%time
muon = df[df.particle_type == 1]
plt.plot(muon['Lextra_X[3]'], muon['Lextra_Y[3]'], 'o')

In [ ]:
len(muon)

In [ ]:
df[['weight', 'sWeight', 'kinWeight']].corr()

In [ ]:
#df[['Lextra_X[0]', 'Mextra_DX2[0]', 'MatchedHit_TYPE[0]', 'MatchedHit_X[0]', 'MatchedHit_DX[0]']]

In [ ]:
uncrossed = df[df['MatchedHit_TYPE[3]'] == 2]
for i in range(0,4):
    print(uncrossed[['MatchedHit_X[{}]'.format(i), 'Lextra_X[{}]'.format(i)]].corr())

In [ ]:
crossed0 = df[df['MatchedHit_TYPE[0]'] == 2]
uncrossed1 = crossed0[crossed0['MatchedHit_TYPE[1]'] == 1]
print(len(crossed0) / len(uncrossed1))

In [ ]:
len(uncrossed1) / len(crossed0) 

In [ ]:
for i in range(1,4):
    print(len(crossed0[crossed0['MatchedHit_TYPE[{}]'.format(i)] == 1]))

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16, 10))
#распределение времени на каждую пластинку

for row, i in zip(axes, range(0,4)):
    row.hist(df['MatchedHit_T[{}]'.format(i)].value_counts())

In [ ]:
#df['MatchedHit_Z[3]'].value_counts()

In [ ]:
# распределение координат по xyz и по пластинам
fig, axes = plt.subplots(nrows=4, ncols=3, figsize=(16, 10))
axis = ['X', 'Y', 'Z']
#df1 = df[abs(df['MatchedHit_X[0]']) < 200]
muon = df1[df1.label == 1]
back = df1[df1.label == 0]

for row, i in zip(axes, range(0,4)):
    for col,ax in zip(row, axis):
        col.hist(muon['MatchedHit_{}[{}]'.format(ax, i)], color = 'g')
        col.hist(back['MatchedHit_{}[{}]'.format(ax, i)], color = 'r', alpha = 0.5)

        


In [ ]:
#df['MatchedHit_X[0]']
df.label.value_counts()

In [ ]:
# среднее при непересеченных и переесеченных
import itertools
mean_uncrossed = []
mean_crossed = []

for hit_type, n_station in itertools.product([1,2], [0,1,2,3]):
    mean = df[df['MatchedHit_TYPE[{}]'.format(n_station)] == hit_type]['MatchedHit_DX[{}]'.format(n_station)].mean()
    if hit_type == 1:
        mean_uncrossed.append(mean)
    else:
        mean_crossed.append(mean)
    print('station {} hit_type {} mean {}'.format(n_station, hit_type, mean))
    

In [ ]:
plt.plot(range(0,4), mean_uncrossed, label = 'непересеченные')
plt.plot(range(0,4), mean_crossed, color = 'r', label = 'пересеченные')
plt.legend();

In [ ]:
df[['MatchedHit_T[0]','MatchedHit_T[1]','MatchedHit_T[2]','MatchedHit_T[3]','MatchedHit_DT[0]','MatchedHit_DT[1]','MatchedHit_DT[2]','MatchedHit_DT[3]']] 

In [ ]:
for i in [0,1,2]:
    cr = df[(df['MatchedHit_TYPE[0]'] == 1) & (df['particle_type'] == i)]
    un = df[(df['MatchedHit_TYPE[0]'] == 2) & (df['particle_type'] == i)]
    print('среднее непересеченное {} для {}'.format(cr['FOI_hits_N'].mean(), i))
    print('среднее пересеченное {} для {}'.format(un['FOI_hits_N'].mean(), i))

In [ ]:
df['FOI_hits_X'][21]

In [ ]:
df['FOI_hits_S'][21]

In [ ]:
df['FOI_hits_N'][21]

In [ ]:
df[['MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]', 'MatchedHit_X[3]']].head(1)

In [ ]:
df['FOI_hits_DX'][21]

In [ ]:
#df[['MatchedHit_X[0]', 'MatchedHit_X[1]', 'MatchedHit_X[2]', 'MatchedHit_X[3]']]


In [ ]:
#df[['ncl[0]', 'avg_cs[0]']]

In [ ]:
pion = df2[df2['particle_type'] == 0]
muon = df2[df2['particle_type'] == 1]
proton = df2[df2['particle_type'] == 2]

In [ ]:
print('pion {} muon {} proton {}'.format(pion.PT.mean(), muon.PT.mean(), proton.PT.mean()))

In [ ]:
plt.hist(pion.PT)
plt.hist(muon.PT, color = 'r', alpha = 0.5)
plt.hist(proton.PT, color = 'g');

In [ ]:
muon['MatchedHit_X[3]'].count()

In [ ]:
pion['MatchedHit_X[3]'].count()

In [ ]:
proton['MatchedHit_X[3]'].count()

In [ ]:
muon[muon['MatchedHit_X[3]'] != -9999].id.count()